# Centralized Solution of Multi-Regional Transmission Expansion Horizontal Investment Coordination Problem

This notebook contains code for a three zone test case. The case consists of an artificial merging of IEEE 14 nodes, IEEE 30 nodes, and a 5 node test system, respectively, for each zone. This notebook assumes there is a single entity (e.g. national planner) to solve the optimization problem centrally.
Results from this notebook serve a benchmark for the decentralized algorithms. 

In [1]:
# activate julia environment in a controlled way
julia_environment = "../central_control_PyJuMP/Julia_src/activate.jl"
include(julia_environment)

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Ch

Activating the Julia virtual environment


In [2]:
# activate required packages
using Pkg
using JuMP
using GLPK
using HiGHS
#using Gurobi
#using CPLEX
#using Xpress
#using Cbc
#using Clp
using XLSX
using CSV
using DataFrames
using PrettyPrint

# print package status
Pkg.status()

Status `~/.julia/environments/v1.8/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
⌃ [336ed68f] CSV v0.8.2
  [9961bab8] Cbc v1.0.3
⌃ [a93c6f00] DataFrames v0.22.7
  [864edb3b] DataStructures v0.18.13
  [60bf3e95] GLPK v1.1.1
  [5c1252a2] GeometryBasics v0.4.5
  [87dc4568] HiGHS v1.4.3
  [7073ff75] IJulia v1.24.0
  [b6b21f68] Ipopt v1.2.0
  [4076af6c] JuMP v1.8.2
  [b8f27783] MathOptInterface v1.13.0
  [fdba3010] MathProgBase v0.7.8
  [91a5bcdd] Plots v1.38.6
  [8162dcfd] PrettyPrint v0.2.0
  [731186ca] RecursiveArrayTools v2.38.0
  [82193955] SCIP v0.11.12
  [2913bbd2] StatsBase v0.33.21
  [fdbf4ff8] XLSX v0.8.4
  [ddb6d928] YAML v0.4.8
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
  [10745b16] Statistics
Info Packages marked with ⌃ have new versions available and may be upgradable.


## Problem set up

Set up the example problem by reading the input data files from the `../Input_Data/` folder, then separate the data frames for each zone.

The input data includes: 

* `CandLine.csv` which contains information on shared _candidate lines_ between zones
* `CandLineInt.csv` which contains information on internal _candidate lines_ within each zone
* `SharedEline.csv` which contains information on existing _shared lines_ between zones
* `Tran.csv` which contains information on existing transmission lines within each zone (i.e. as defined in the cited IEEE test systems)
* `Gen.csv` which contains information on the generators as defined in the cited IEEE test systems
* `Load.csv` which contains information on the loads as defined in the cited IEEE test systems

After loading the input data, construct several DataFrames for each zone, i.e. for each set of lines, generators, loads, etc.


In [3]:
# Read input files to set the problem
data_path = "../Input_Data/"

# note that network data are convertered xlsx data (not true csv files)
shared_cand = DataFrame(XLSX.readtable(data_path * "CandLine.csv", "Taul1", header=true)) # shared candidate lines
int_cand =  DataFrame(XLSX.readtable(data_path * "CandLineInt.csv", "Taul1", header=true)) # internal candidate lines
shared_ex =  DataFrame(XLSX.readtable(data_path * "SharedEline.csv", "Taul1", header=true)) # shared existing lines
int_ex = DataFrame(XLSX.readtable(data_path * "Tran.csv", "Taul1", header=true)) # internal existing lines
gen =  DataFrame(XLSX.readtable(data_path * "Gen.csv" , "Taul1", header=true)) # generator data
load =  DataFrame(XLSX.readtable(data_path * "Load.csv", "Taul1", header=true)) # load data

# read scenario and zone summary data. These are true CSV files, not XLSX
scen_prob = CSV.read(data_path * "Scenario_Probability.csv", DataFrame) # scenario probabilities
zone_summary = CSV.read(data_path * "Zone_Summary.csv", DataFrame) # region-number of nodes

,Zone,Nodes_Total
,Int64,Int64
1,1,14
2,2,30
3,3,5


In [4]:
# Functionally define dataframes for each element (loads, generators, etc.) within each zone
# The left hand side is the singleton function that references the dataframe on the right hand side, e.g. l(1) is the load dataframe for zone 1


l(i,s) = load[load.zoneNum .== i, [1,2,s+2]] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1.* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i
scen_weight(s) = scen_prob.scen_weight[scen_prob.scenario .== s,:]

scen_weight (generic function with 1 method)

## Build the Optimization Problem in JuMP

Initialize the JuMP model object by calling `Model(ENGING.Optimizer)` where `ENGINE` is your optimization engine of choice, e.g. Gurobi, CPLEX, Xpress, HiGHS, etc. The optimization engine must have a solver for mixed integer linear programing (MIP). 

After initializing the model, we will add the variables, constraints, and objective function. 
Decision variables for the model (and their variable type) include:
> * Generator dispatch (float)
> * Decision on candidate shared lines (binary)
> * Decision on candidate internal lines (binary)
> * Power flow (float) and phase angle (float) on candidate shared lines
> * Power flow (float) and phase angle (float) on candidate internal lines
> * Power flow (float) and phase angle (float) on existing shared lines
> * Power flow (float) and phase angle (float) on existing internal lines

The objective function is the `total_cost` to build new lines and dispatch generation. The cost is defined as:
> \[EQUATION GOES HERE\]

The constraints include: 
> \[EQUATIONS GO HERE\]

In [ ]:
# initialize the JuMP model object and set the optimization engine
# solvers = [Gurobi.Optimizer, Clp.Optimzer, Cbc.Optimizer, HiGHS.Optimizer, Ipopt.Optimizer, CPLEX.Optimizer, Xpress.Optimizer]

Mod3 = Model(GLPK.Optimizer) #use different solver e.g. IPOPT, GLPK

In [ ]:
zone_number = 3; #number of zones to consider

In [ ]:
@variable(Mod,0 <= gen_var_1[1:14,1:nrow(scen_prob)]) # generation at node 1
@variable(Mod,0 <= gen_var_2[1:30,1:nrow(scen_prob)]) # generation at node 2
@variable(Mod,0 <= gen_var_3[1:5,1:nrow(scen_prob)]) # generation at node 3

@variable(Mod,shared_line_decision_var[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines

@variable(Mod,int_line_decision_var_1[1:nrow(int_c(1))], Bin) #Decision variable for internal candidate lines of zone 1 (Binary)
@variable(Mod,int_line_decision_var_2[1:nrow(int_c(2))], Bin) #Decision variable for internal candidate lines of zone 2 (Binary)
@variable(Mod,int_line_decision_var_3[1:nrow(int_c(3))], Bin) #Decision variable for internal candidate lines of zone 3 (Binary)

@variable(Mod,shared_cand_flow[1:nrow(shared_cand),1:nrow(scen_prob)])  #Power flowing on shared candidate lines 

@variable(Mod, int_cand_flow_1[1:nrow(int_c(1)),1:nrow(scen_prob)]) #Power flowing on internal candidate lines of zone 1 
@variable(Mod, int_cand_flow_2[1:nrow(int_c(2)),1:nrow(scen_prob)]) #Power flowing on internal candidate lines of zone 2
@variable(Mod, int_cand_flow_3[1:nrow(int_c(3)),1:nrow(scen_prob)]) #Power flowing on internal candidate lines of zone 3

@variable(Mod, 0 <= node_voltage_phase_angle[i=1:nrow(zone_summary),j=1:zone_summary.Nodes_Total[i], 1:nrow(scen_prob)]<= 2*pi)

@variable(Mod, shared_ex_flow[1:nrow(shared_ex),1:nrow(scen_prob)])  #Power flowing on shared existing shared lines

@variable(Mod, int_ex_flow_1[1:nrow(int_e(1)),1:nrow(scen_prob)])  #Power flowing on internal existing lines of zone 1 
@variable(Mod, int_ex_flow_2[1:nrow(int_e(2)),1:nrow(scen_prob)])  #Power flowing on internal existing lines of zone 2
@variable(Mod, int_ex_flow_3[1:nrow(int_e(3)),1:nrow(scen_prob)])  #Power flowing on internal existing lines of zone 3


In [ ]:
@expression(Mod,total_cost,0)

In [ ]:
@expression(Mod, gen_costNS1[n=1:14,s=1:nrow(scen_prob)], (gen_var_1[n,s] .* sum((g(1).gNodeID .== n) .* MC(1))))
@expression(Mod, gen_cost_prob_weightedS1[s=1:nrow(scen_prob)], (scen_weight(s)[1]).*sum(gen_costNS1[n,s] for n in 1:14))

In [ ]:
@expression(Mod, gen_cost_expected1, sum(gen_cost_prob_weightedS1[s] for s in 1:nrow(scen_prob)))
Mod[:total_cost] += gen_cost_expected1

In [ ]:
@expression(Mod, gen_costNS2[n=1:30,s=1:nrow(scen_prob)], (gen_var_2[n,s] .* sum((g(2).gNodeID .== n) .* MC(2))))

In [ ]:
@expression(Mod, gen_cost_prob_weightedS2[s=1:nrow(scen_prob)], (scen_weight(s)[1]).*sum(gen_costNS2[n,s] for n in 1:30))

In [ ]:
@expression(Mod, gen_cost_expected2, sum(gen_cost_prob_weightedS2[s] for s in 1:nrow(scen_prob)))

In [ ]:
Mod[:total_cost]+=gen_cost_expected2

In [ ]:
@expression(Mod, gen_costNS3[n=1:5,s=1:nrow(scen_prob)], (gen_var_3[n,s] .* sum((g(3).gNodeID .== n) .* MC(3))))
@expression(Mod, gen_cost_prob_weightedS3[s=1:nrow(scen_prob)], (scen_weight(s)[1]).*sum(gen_costNS3[n,s] for n in 1:5))

In [ ]:
@expression(Mod, gen_cost_expected3, sum(gen_cost_prob_weightedS3[s] for s in 1:nrow(scen_prob)))

In [ ]:
Mod[:total_cost]+=gen_cost_expected3

In [ ]:
@expression(Mod, investment_cost , (sum(shared_line_decision_var[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
            .+ sum(int_line_decision_var_1[c] .* int_c(1).costPerCap[c] .* int_c(1).interestRate[c] 
            .*((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c]) ./ (((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c])-1) for c in 1:nrow(int_c(1)))
            .+ sum(int_line_decision_var_2[c] .* int_c(2).costPerCap[c] .* int_c(2).interestRate[c] 
            .*((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c]) ./ (((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c])-1) for c in 1:nrow(int_c(2)))
            .+ sum(int_line_decision_var_3[c] .* int_c(3).costPerCap[c] .* int_c(3).interestRate[c] 
            .*((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c]) ./ (((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c])-1) for c in 1:nrow(int_c(3)))))
Mod[:total_cost]+=investment_cost

In [ ]:
#The first zone
for n in 1:14
    for s in 1:nrow(scen_prob)
        # Power balance constraint for each node
        @constraint(Mod, sum(g(1).gNodeID .== n) .* gen_var_1[n,s] .+ sum(l(1,s)[:,3].* (l(1,s).lNodeID .== n))./100 .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(1) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(1) .* shared_cand_flow[:,s]) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(1) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(1) .* shared_ex_flow[:,s]) .+
        sum((int_c(1).tNodeID1 .== n) .* int_cand_flow_1[:,s]) .- sum((int_c(1).tNodeID2 .== n) .* int_cand_flow_1[:,s]) .+
        sum((int_e(1).tNodeID1 .== n) .* int_ex_flow_1[:,s]) .- sum((int_e(1).tNodeID2 .== n) .* int_ex_flow_1[:,s]))
        #Lower limit for generation of each node
        @constraint(Mod, sum(g(1).gNodeID .== n) .* gen_var_1[n,s] .<= sum((g(1).gNodeID .== n) .* g(1).PgMax)./100)
        #Upper limit for generation of each node
        @constraint(Mod, sum((g(1).gNodeID .== n) .* g(1).PgMin)./100 .<= sum(g(1).gNodeID .== n) .* gen_var_1[n,s])
    end
end

In [ ]:
#The second zone
for n in 1:30
    for s in 1:nrow(scen_prob)
        # Power balance constraint for each node
        @constraint(Mod, sum(g(2).gNodeID .== n) .* gen_var_2[n,s] .+ sum(l(2,s)[:,3] .* (l(2,s).lNodeID .== n))./100 .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(2) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(2) .* shared_cand_flow[:,s]) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(2) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(2) .* shared_ex_flow[:,s]) .+
        sum((int_c(2).tNodeID1 .== n) .* int_cand_flow_2[:,s]) .- sum((int_c(2).tNodeID2 .== n) .* int_cand_flow_2[:,s]) .+
        sum((int_e(2).tNodeID1 .== n) .* int_ex_flow_2[:,s]) .- sum((int_e(2).tNodeID2 .== n) .* int_ex_flow_2[:,s]))
        #Lower limit for generation of each node
        @constraint(Mod, sum(g(2).gNodeID .== n) .* gen_var_2[n,s] .<= sum((g(2).gNodeID .== n) .* g(2).PgMax)./100)
        #Upper limit for generation of each node
        @constraint(Mod, sum((g(2).gNodeID .== n) .* g(2).PgMin)./100 .<= sum(g(2).gNodeID .== n) .* gen_var_2[n,s])
    end
end

In [ ]:
#The third zone
for n in 1:5
    for s in 1:nrow(scen_prob)
        # Power balance constraint for each node
        @constraint(Mod, sum(g(3).gNodeID .== n) .* gen_var_3[n,s] .+ sum(l(3,s)[:,3] .* (l(3,s).lNodeID .== n))./100 .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(3) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(3) .* shared_cand_flow[:,s]) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(3) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(3) .* shared_ex_flow[:,s]) .+
        sum((int_c(3).tNodeID1 .== n) .* int_cand_flow_3[:,s]) .- sum((int_c(3).tNodeID2 .== n) .* int_cand_flow_3[:,s]) .+
        sum((int_e(3).tNodeID1 .== n) .* int_ex_flow_3[:,s]) .- sum((int_e(3).tNodeID2 .== n) .* int_ex_flow_3[:,s]))
        #Lower limit for generation of each node
        @constraint(Mod, sum(g(3).gNodeID .== n) .* gen_var_3[n,s] .<= sum((g(3).gNodeID .== n) .* g(3).PgMax)./100)
        #Upper limit for generation of each node
        @constraint(Mod, sum((g(3).gNodeID .== n) .* g(3).PgMin)./100 .<= sum(g(3).gNodeID .== n) .* gen_var_3[n,s])
    end
end

In [ ]:
#Shared candidate lines
M = 5 # this should be changed/translated to PU value...
for c in 1:nrow(shared_cand)
    for s in 1:nrow(scen_prob)
        @constraint(Mod,-M .* (1 .- shared_line_decision_var[c]) .<= shared_cand_flow[c,s] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s])))
        @constraint(Mod, shared_cand_flow[c,s] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s])) .<= M .* (1 .- shared_line_decision_var[c]))
        #limiting the upper bound of power flow flowing within candidate shared lines
        @constraint(Mod, shared_cand_flow[c,s] .<= shared_line_decision_var[c] .*shared_cand.ptMax[c]./100)
        #Limiting the lower bound of power flowing within the candidate shared lines
        @constraint(Mod, -(shared_cand.ptMax[c]./100) .* shared_line_decision_var[c] .<= shared_cand_flow[c,s])
    end
end

In [ ]:
#Shared existing lines
for h in 1:nrow(shared_ex)
    for s in 1:nrow(scen_prob)
        @constraint(Mod, shared_ex_flow[h,s] .== (1 ./ shared_ex.reacT[h]) .* (node_voltage_phase_angle[shared_ex.nodeZone1[h],shared_ex.tNodeID1[h],s] .- node_voltage_phase_angle[shared_ex.nodeZone2[h],shared_ex.tNodeID2[h],s]))
        @constraint(Mod, shared_ex_flow[h,s] .<= shared_ex. ptMax[h]./100)
        @constraint(Mod, -shared_ex.ptMax[h]./100 .<= shared_ex_flow[h,s])
    end
end

In [ ]:
#Zone 1 internal candidate lines
for c in 1:nrow(int_c(1))
    for s in 1:nrow(scen_prob)
        @constraint(Mod,-M .* (1 .- int_line_decision_var_1[c]) .<= int_cand_flow_1[c,s] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s])))
        @constraint(Mod, int_cand_flow_1[c,s] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s])) .<= M .* (1 .- int_line_decision_var_1[c]))
        @constraint(Mod, int_cand_flow_1[c,s] .<= int_line_decision_var_1[c] .* int_c(1).ptMax[c]./100)
        @constraint(Mod, -(int_c(1).ptMax[c]./100) .*int_line_decision_var_1[c] .<= int_cand_flow_1[c,s])
    end
end

In [ ]:
#Zone 2 internal candidate lines
for c in 1:nrow(int_c(2))
    for s in 1:nrow(scen_prob)
        @constraint(Mod,-M .* (1 .- int_line_decision_var_2[c]) .<= int_cand_flow_2[c,s] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s])))
        @constraint(Mod, int_cand_flow_2[c,s] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s])) .<= M .* (1 .- int_line_decision_var_2[c]))
        @constraint(Mod, int_cand_flow_2[c,s] .<= int_line_decision_var_2[c] .* int_c(2).ptMax[c]./100)
        @constraint(Mod, -(int_c(2).ptMax[c]./100) .*int_line_decision_var_2[c] .<= int_cand_flow_2[c,s])
    end
end

In [ ]:
#Zone 3 internal candidate lines
for c in 1:nrow(int_c(3))
    for s in 1:nrow(scen_prob)
        @constraint(Mod,-M .* (1 .- int_line_decision_var_3[c]) .<= int_cand_flow_3[c,s] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s])))
        @constraint(Mod, int_cand_flow_3[c,s] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s])) .<= M .* (1 .- int_line_decision_var_3[c]))
        @constraint(Mod, int_cand_flow_3[c,s] .<= int_line_decision_var_3[c] .* int_c(3).ptMax[c]./100)
        @constraint(Mod, -(int_c(3).ptMax[c]./100) .*int_line_decision_var_3[c] .<= int_cand_flow_3[c,s])
    end
end

In [ ]:
#Zone 1 internal existing lines
for h in 1:nrow(int_e(1))
    for s in 1:nrow(scen_prob)
        @constraint(Mod, int_ex_flow_1[h,s] .== (1 ./ int_e(1).reacT[h]) .* (node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID1[h],s] .- node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID2[h],s]))
        @constraint(Mod, int_ex_flow_1[h,s] .<= int_e(1).ptMax[h]./100)
        @constraint(Mod, -int_e(1).ptMax[h]./100 .<= int_ex_flow_1[h,s])
    end
end

In [ ]:
#Zone 2 internal existing lines
for h in 1:nrow(int_e(2))
    for s in 1:nrow(scen_prob)
        @constraint(Mod, int_ex_flow_2[h,s] .== (1 ./ int_e(2).reacT[h]) .* (node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID1[h],s] .- node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID2[h],s]))
        @constraint(Mod, int_ex_flow_2[h,s] .<= int_e(2).ptMax[h]./100)
        @constraint(Mod, -int_e(2).ptMax[h]./100 .<= int_ex_flow_2[h,s])
    end
end

In [ ]:
#Zone 3 internal existing lines
for h in 1:nrow(int_e(3))
    for s in 1:nrow(scen_prob)
        @constraint(Mod, int_ex_flow_3[h,s] .== (1 ./ int_e(3).reacT[h]) .* (node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID1[h],s] .- node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID2[h],s]))
        @constraint(Mod, int_ex_flow_3[h,s] .<= int_e(3).ptMax[h]./100)
        @constraint(Mod, -int_e(3).ptMax[h]./100 .<= int_ex_flow_3[h,s])
    end
end

In [ ]:
@objective(Mod, Min, Mod[:total_cost])
optimize!(Mod)

In [ ]:
termination_status(Mod)

In [ ]:
if termination_status(Mod) == MOI.OPTIMAL
    shared_line_decision = value.(shared_line_decision_var)
    shared_cand_power = (value.(shared_cand_flow)) .* 100
    node_voltage_phase_angle = value.(node_voltage_phase_angle)
    shared_ex_power = (value.(shared_ex_flow)) .* 100
    int_line_decision_1 = value.(int_line_decision_var_1)
    int_line_decision_2 = value.(int_line_decision_var_2)
    int_line_decision_3 = value.(int_line_decision_var_3)
    int_line_flow_1 = (value.(int_cand_flow_1)) .* 100
    int_line_flow_2 = (value.(int_cand_flow_2)) .* 100
    int_line_flow_3 = (value.(int_cand_flow_3)) .* 100
    generation_1 = (value.(gen_var_1)) .* 100
    generation_2 = (value.(gen_var_2)) .* 100
    generation_3 = (value.(gen_var_3)) .* 100
    internal_existing_line_flow_1 = (value.(int_ex_flow_1)) .* 100
    internal_existing_line_flow_2 = (value.(int_ex_flow_2)) .* 100
    internal_existing_line_flow_3 = (value.(int_ex_flow_3)) .* 100
    obj_value = objective_value(Mod)
end

In [ ]:
shared_line_decision

In [ ]:
shared_cand_power

In [ ]:
node_voltage_phase_angle

In [ ]:
shared_ex_power

In [ ]:
int_line_decision_1

In [ ]:
int_line_decision_2

In [ ]:
int_line_decision_3

In [ ]:
obj_value

In [ ]:
generation_1

In [ ]:
show(stdout, "text/plain", generation_2)

In [ ]:
generation_3

In [ ]:
int_line_flow_1

In [ ]:
int_line_flow_2

In [ ]:
int_line_flow_3

In [ ]:
internal_existing_line_flow_1

In [ ]:
internal_existing_line_flow_2

In [ ]:
internal_existing_line_flow_3